In [4]:
import time
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine, inspect
from polygon import RESTClient


def get_real_time_forex(client, ticker):
    from_currency, to_currency = ticker[2:5], ticker[5:]
    quote = client.get_real_time_currency_conversion(from_=from_currency, to=to_currency, amount=1, precision=4)
    return {
        "timestamp": quote.last.timestamp,
        "fx_rate": quote.converted,
    }


def store_real_time_forex_data(ticker, engine):
    data = get_real_time_forex(client, ticker)
    df = pd.DataFrame([data])
    df.to_sql(f'{ticker}_auxiliary', engine, if_exists='replace', index=False)


def get_past_6_minute_data(ticker, engine):
    query = f'SELECT * FROM {ticker}_auxiliary'
    df = pd.read_sql_query(query, engine)
    return df


def store_summary_data(ticker, summary, engine):
    df = pd.DataFrame([summary])
    df.to_sql(f'{ticker}_final', engine, if_exists='append', index=False)


def clear_auxiliary_database(engine):
    table_names = get_table_names(engine)
    auxiliary_table_names = [t for t in table_names if "auxiliary" in t]
    with engine.begin() as connection:
        for table_name in auxiliary_table_names:
            connection.execute(f'DELETE FROM {table_name}')


def get_table_names(engine):
    inspector = inspect(engine)
    return inspector.get_table_names()


def read_final_data_and_export_csv(ticker, engine):
    query = f'SELECT * FROM {ticker}_final'
    df = pd.read_sql_query(query, engine)
    df.to_csv(f'{ticker}_final.csv', index=False)


api_key = "your_api_key_here"
client = RESTClient(api_key=api_key)
tickers = ["C:EURUSD", "C:USDCAD", "C:USDCHF"]

auxiliary_engine = create_engine('sqlite:///auxiliary.db')
final_engine = create_engine('sqlite:///final.db')

def excute():
    for i in range(3600):  # 5 hours
#         time.sleep(1)
        

        for ticker in tickers:
            store_real_time_forex_data(ticker, auxiliary_engine)

        if i % 360 == 0 and i > 0:
            print(i)
            for ticker in tickers:
                df = get_past_6_minute_data(ticker, auxiliary_engine)

                max_rate = df['fx_rate'].max()
                min_rate = df['fx_rate'].min()
                mean_rate = df['fx_rate'].mean()
                vol = (max_rate - min_rate) / mean_rate if mean_rate != 0 else 0

                crosses = 0
                for n in range(1, 101):
                    upper_band = mean_rate + n * 0.025 * vol
                    lower_band = mean_rate - n * 0.025 * vol

                    for index in range(1, df.shape[0]):
                        prev_rate = df.loc[index - 1, 'fx_rate']
                        current_rate = df.loc[index, 'fx_rate']

                        if (prev_rate < upper_band < current_rate) or (prev_rate > upper_band > current_rate):
                            crosses += 1

                        if (prev_rate < lower_band < current_rate) or (prev_rate > lower_band > current_rate):
                            crosses += 1

                if i > 360:  # Start calculating FD from period 2
                    
                    if (max_rate - min_rate) != 0:
                        fd = crosses / (max_rate - min_rate)
                    else:
                        fd = 0
                else:
                    fd = 0

                summary = {
                    "timestamp": df.loc[0, 'timestamp'],
                    "max": max_rate,
                    "min": min_rate,
                    "mean": mean_rate,
                    "vol": vol,
                    "fd": fd
                }

                store_summary_data(ticker, summary, final_engine)

            clear_auxiliary_database(auxiliary_engine)

    for ticker in tickers:
        read_final_data_and_export_csv(ticker, final_engine)



                    
                    


In [5]:
excute()

BadResponse: {"status":"ERROR","request_id":"a22d7f2ecc533201e15cc9c205f79884","error":"Unknown API Key"}